In [1]:
import sys 
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'statistics_server'))

from opensearch_client import OpensearchClient
from psql_client import PSQLClient

sql_client = PSQLClient(
	host='localhost',
	port=5432,
	user='default',
	password='12345',
	dbname='hack',
)

client = OpensearchClient(
	host='localhost',
	port=9200,
	user='admin',
	password='admin',
	sql_client=sql_client,
)

In [2]:
client.create_restrictions_index(True)

In [3]:
client.search_restrictions('мясо')

[{'entity_id': '672525085',
  'okpd2_code': '',
  'name': 'Консервы из мяса и субпродуктов птицы, мясо-растительные и растительно-мясные с использованием мяса птицы для детского питания',
  'doughters': '',
  'restrictions': 'Лот относится к аукционной продукции.\nДопустимые способы определения поставщика: электронный аукцион, электронный запрос котировок, электронный запрос предложений, закрытый аукцион и закупка у единственного поставщика. Правило: закупка осуществляется в соответствии с 44 ФЗ,  не офсет, не муниципальный заказчик. \n'}]

In [3]:
print(client.client.cat.indices('contracts'))

yellow open contracts XNBXhjNbRd6L-OLtK-89zQ 1 1 1107 0 305.6kb 305.6kb



### Метч таблиц balances_data и procurement_contracts

In [ ]:
from tqdm import tqdm

balances_rows = client.sql_client.select('select id, item_name from balances_account')
balances_procurement_contracts_match = []
for row in tqdm(balances_rows):
	contracts_match = client.search_contract_subject(row['item_name'], 2)
	if not contracts_match:
		balances_procurement_contracts_match.append({
			'balances_id': row['id'],
			'balances_item_name': row['item_name'],
			'matched': False,
		})
		continue
	
	match_row = {
		'balances_id': row['id'],
		'balances_item_name': row['item_name'],
		'contracts_contract_id': contracts_match[0]['contract_id'],
		'contracts_lot_number': contracts_match[0]['lot_number'],
		'contracts_spgz_name': contracts_match[0]['spgz_name'],
		'contracts_contract_subject': contracts_match[0]['contract_subject'],
		'matched': True,
	}
	if len(contracts_match) > 1:
		match_row['contracts_contract_id_2_match'] = contracts_match[1]['contract_id']
		match_row['contracts_lot_number_2_match'] = contracts_match[1]['lot_number']
	else:
		match_row['contracts_contract_id_2_match'] = ''
		match_row['contracts_lot_number_2_match'] = ''

	balances_procurement_contracts_match.append(match_row)

In [ ]:
print(len(balances_procurement_contracts_match))
print(len([t for t in balances_procurement_contracts_match if t['matched']]))

In [ ]:
import pandas as pd
df = pd.DataFrame(balances_procurement_contracts_match)
df.head(2)

In [ ]:
# CREATE TABLE public.match_contr_bal (
# 	balances_id text NULL,
# 	balances_item_name text NULL,
# 	contracts_contract_id text NULL,
# 	contract_lot_number text NULL,
# 	contracts_spgz_name text NULL,
# 	contracts_contract_subject text NULL,
# 	contracts_contract_id_2_match text NULL,
# 	contracts_lot_number_2_match text NULL,
# 	"matched" bool NULL
# );
cursor = client.sql_client.client.cursor()
for row in tadm(balances_procurement_contracts_match):
	cursor.execute(f"""
	insert into match_contr_bal values (
		'{row.get('balances_id', '')}',
		'{row.get('balances_item_name', '')}',
		'{row.get('contracts_contract_id', '')}',
		'{row.get('contracts_lot_number', '')}',
		'{row.get('contracts_spgz_name', '')}',
		'{row.get('contracts_contract_subject', '')}',
		'{row.get('contracts_contract_id_2_match', '')}',
		'{row.get('contracts_lot_number_2_match', '')}',
		{row.get('matched')}
	)
	""")
client.sql_client.client.commit()
cursor.close()

In [ ]:
client.sql_client.client.commit()

### Метч таблиц balances_account и procurement_contracts

In [ ]:
from tqdm import tqdm

financial_rows = client.sql_client.select('select id, item_name from financial_data')
financian_procurement_contracts_match = []
for row in tqdm(financial_rows):
	contracts_match = client.search_contract_subject(row['item_name'], 2)
	if not contracts_match:
		financian_procurement_contracts_match.append({
			'financial_id': row['id'],
			'financial_item_name': row['item_name'],
			'matched': False,
		})
		continue
	
	match_row = {
		'financial_id': row['id'],
		'financial_item_name': row['item_name'],
		'contracts_contract_id': contracts_match[0]['contract_id'],
		'contracts_lot_number': contracts_match[0]['lot_number'],
		'contracts_spgz_name': contracts_match[0]['spgz_name'],
		'contracts_contract_subject': contracts_match[0]['contract_subject'],
		'matched': True,
	}
	if len(contracts_match) > 1:
		match_row['contracts_contract_id_2_match'] = contracts_match[1]['contract_id']
		match_row['contracts_lot_number_2_match'] = contracts_match[1]['lot_number']
	else:
		match_row['contracts_contract_id_2_match'] = ''
		match_row['contracts_lot_number_2_match'] = ''

	financian_procurement_contracts_match.append(match_row)

In [ ]:
print(len(financian_procurement_contracts_match))
print(len([t for t in financian_procurement_contracts_match if t['matched']]))

In [ ]:
import pandas as pd
df = pd.DataFrame(financian_procurement_contracts_match)
df.head(2)

In [ ]:
# CREATE TABLE public.match_contr_fin (
# 	financial_id text NULL,
# 	financial_item_name text NULL,
# 	contracts_contract_id text NULL,
# 	contract_lot_number text NULL,
# 	contracts_spgz_name text NULL,
# 	contracts_contract_subject text NULL,
# 	contracts_contract_id_2_match text NULL,
# 	contracts_lot_number_2_match text NULL,
# 	"matched" bool NULL
# );
cursor = client.sql_client.client.cursor()
for row in financian_procurement_contracts_match:
	cursor.execute(f"""
	insert into match_contr_fin values (
		'{row.get('financial_id', '')}',
		'{row.get('financial_item_name', '')}',
		'{row.get('contracts_contract_id', '')}',
		'{row.get('contracts_lot_number', '')}',
		'{row.get('contracts_spgz_name', '')}',
		'{row.get('contracts_contract_subject', '')}',
		'{row.get('contracts_contract_id_2_match', '')}',
		'{row.get('contracts_lot_number_2_match', '')}',
		{row.get('matched')}
	)
	""")
client.sql_client.client.commit()
cursor.close()

In [ ]:
client.sql_client.client.commit()

In [ ]:
[t for t in financian_procurement_contracts_match if not t['matched']]